# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [ ]:
%pip install iso3166

### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [ ]:
%pip install --upgrade plotly

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.12.0)


### Import Statements

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



### Notebook Presentation

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [ ]:
df_data = pd.read_csv('mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [ ]:
print(f'Shape: {df_data.shape}\n'
      f'Rows: {df_data.shape[0]}, '
      f'Columns: {df_data.shape[1]}\n'
      f'Column Names: {df_data.columns}')

In [ ]:
print(f"NaN Values: {df_data.isnull().values.sum()}\n "
      f"Duplicates: {df_data.duplicated().any()}")

## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [ ]:
df_data.drop(df_data.columns[[0, 1]], axis=1, inplace=True)

In [ ]:
df_data['Price'] = df_data.Price.replace(',', '', regex=True)

price_mean = df_data.Price.mean()
df_data['Price'].fillna(price_mean, inplace=True)

## Descriptive Statistics

In [ ]:
df_data.head()

In [ ]:
df_data.info()

# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [ ]:
print(df_data['Organisation'].value_counts())

In [ ]:
sns.displot(df_data, x=df_data.Organisation, aspect=4)
plt.xticks(rotation=90)
plt.show()

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned? 

In [ ]:
print(df_data.Mission_Status.value_counts())

# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [ ]:
print(df_data.Rocket_Status.value_counts())

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

In [ ]:
sns.displot(x=df_data['Price'], bins=90, aspect=4)
plt.xticks(rotation=90)
plt.show()

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [ ]:
country_name = []
for item in df_data.Location:
  country_name.append(item.split()[-1])
country = pd.DataFrame({'Country':country_name})
data2 = pd.concat([df_data, country.reindex(df_data.index)], axis=1)
replaced_data=data2.replace(['Russia', 'Zealand', 'Iran', 'Site', 'Sea', 'Korea', 'Facility', 'Ocean', 'Canaria'], ['Russian Federation', 'New Zealand', 'IRN', 'IRN', 'China', 'Korea, Republic of', 'USA', 'Russian Federation', 'USA'])

alpha3=[]
for item in replaced_data.Country:
  r=countries.get(item).alpha3
  alpha3.append(r)
replaced_data['iso']=alpha3

launches_by_country=[]
for item in replaced_data.iso.unique():
    launches_by_country.append(replaced_data['iso'].value_counts()[item])

In [ ]:
fig = px.choropleth(locations=replaced_data.iso.unique(), color=launches_by_country, hover_name=replaced_data.iso.unique(), color_continuous_scale="matter", )
fig.show()

# Use a Choropleth Map to Show the Number of Failures by Country


In [ ]:
data_failures = replaced_data[replaced_data.Mission_Status!='Success' ]

failure_by_country=[]
for item in data_failures.iso.unique():
    failure_by_country.append(data_failures['iso'].value_counts()[item])

In [ ]:
figure = px.choropleth(locations=data_failures.iso.unique(), color=failure_by_country, hover_name=data_failures.iso.unique(), color_continuous_scale="matter")
figure.show()

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

In [ ]:
figure = px.sunburst(replaced_data, path=['Country', 'Organisation', 'Mission_Status'])
figure.show()

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

In [ ]:
replaced_data.groupby(by='Organisation').sum()

# Analyse the Amount of Money Spent by Organisation per Launch

In [ ]:
replaced_data.groupby(by='Organisation').mean()

# Chart the Number of Launches per Year

In [ ]:
y = []
for item in replaced_data.Date:
    y.append(item.split(' ')[3])
year = pd.DataFrame({'year':y})
data_year_replaced = pd.concat([replaced_data, year.reindex(replaced_data.index)], axis=1)

count_by_year=[]
for item in data_year_replaced.year.unique():
    count_by_year.append(data_year_replaced['year'].value_counts()[item])


In [ ]:
figure = px.bar(data_year_replaced, x=data_year_replaced.year.unique(), y=count_by_year)
figure.show()

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart. 

In [ ]:
m = []
for item in data_year_replaced.Date():
    m.append(item.split(' ')[1])
month = pd.DataFrame({'month': m})
data_month_replaced = pd.concat([data_year_replaced, month.reindex(data_year_replaced.index)], axis=1)

In [ ]:
sns.displot(data=data_month_replaced, x=data_month_replaced['month'], aspect=2)
plt.show()

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

In [ ]:
data_success = data_month_replaced[data_month_replaced.Mission_Status == 'Success']
print(data_success.month.value_counts())

# How has the Launch Price varied Over Time? 

Create a line chart that shows the average price of rocket launches over time. 

In [ ]:
mean_price_by_time = data5.groupby(by='year').mean()

In [ ]:
plt.figure(figsize=(15, 7))
plt.xticks(rotation=90)
display(plt.plot(mean_price_by_time))

# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

In [ ]:
data_month_replaced.Organisation.value_counts()

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991. 

In [ ]:
I=min(data_month_replaced[data_month_replaced['year']=='1991'].index)
data_coldwar = data_month_replaced.iloc[I:]

launches_by_usa = data_coldwar['Country'].value_counts()['USA']
ussr = ['Armenia', 'Azerbaijan', 'Belarus', 'Estonia', 'Georgia', 'Kazakhstan', 'Kyrgyzstan' ,'Latvia', 'Lithuania' ,'Moldova' ,'Russia' ,'Tajikistan', 'Turkmenistan', 'Ukraine', 'Uzbekistan']
launches_by_ussr = 0
for item in ussr:
  try:
    launches_by_ussr += data_coldwar['Country'].value_counts()[item]
  except KeyError:
    pass

  print(f"Launches of USA:{launches_by_usa} USSR:{launches_by_ussr}")

data_coldwar_success= data_coldwar[data_coldwar.Mission_Status=='Success']
success_by_ussr=0
for i in ussr:
  try:
    success_by_ussr += data_coldwar_success['Country'].value_counts()[i]
  except KeyError:
    pass

In [ ]:
print(f"Successful Launches of USA:{data_coldwar_success['Country'].value_counts()['USA']} USSR:{success_by_ussr}")

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches. 

In [ ]:
figure = px.pie(values=[launches_by_usa, launches_by_ussr], names=['USA', 'USSR'], title="Launches")
figure.show()

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

In [ ]:
data_coldwar_usa= data_coldwar[data_coldwar.Country=='USA']
coldwar_count_by_year_usa = []
for item in data_coldwar_usa.year.unique():
  coldwar_count_by_year_usa.append(data_coldwar_usa['year'].value_counts()[item])

data_coldwar_ussr=pd.DataFrame()
for item in ussr:
  try:
    d=data_coldwar[data_coldwar.Country==item]
    data_coldwar_ussr=data_coldwar_ussr.append(d)
  except KeyError:
    pass
coldwar_count_by_year_ussr = []
for item in data_coldwar_ussr.year.unique():
  coldwar_count_by_year_ussr.append(data_coldwar_ussr['year'].value_counts()[item])

In [ ]:
figure = px.bar(x=data_coldwar_usa.year.unique(), y=coldwar_count_by_year_usa, labels={'x':'year','y':'number of launches'},title="Laucnches of USA vs USSR by year")
figure.add_bar(x=data_coldwar_ussr.year.unique(),y=coldwar_count_by_year_ussr,name="USSR")
figure.show()

## Chart the Total Number of Mission Failures Year on Year.

In [ ]:
data_failure_usa=data_coldwar_usa[data_coldwar_usa.Mission_Status!='Success']
data_failure_ussr=data_coldwar_ussr[data_coldwar_ussr.Mission_Status!='Success']

fail_count_by_year_usa = []
for item in data_failure_usa.year.unique():
  fail_count_by_year_usa.append(data_failure_usa['year'].value_counts()[item])

fail_count_by_year_ussr = []
for item in data_failure_ussr.year.unique():
  fail_count_by_year_ussr.append(data_failure_ussr['year'].value_counts()[item])

In [ ]:
figure = px.bar( x=data_failure_usa.year.unique(), y=fail_count_by_year_usa, labels={'x':'year','y':'number of launches'},title="Failure of USA vs USSR by year")
figure.add_bar(x=data_failure_ussr.year.unique(),y=fail_count_by_year_ussr,name="USSR")
figure.show()

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

In [ ]:
percent_failures_ussr=[]
for item in range(len(coldwar_count_by_year_ussr)):
  y=data_coldwar_ussr.year.unique()[item]
  if y not in data_failure_ussr.year.unique():
    fail_count_by_year_ussr.insert(item, 0)
    r = 0
  else:
    r=round((fail_count_by_year_ussr[item]/coldwar_count_by_year_ussr[item])*100,2)
  percent_failures_ussr.append(r)

In [ ]:
percent_failures_usa=[]
for item in range(len(coldwar_count_by_year_usa)):
  y=data_coldwar_usa.year.unique()[item]
  if y not in data_failure_usa.year.unique():
    fail_count_by_year_usa.insert(item, 0)
    r=0
  else:
    r=round((fail_count_by_year_usa[item]/coldwar_count_by_year_usa[item])*100,2)
  percent_failures_usa.append(r)

In [ ]:
figure1 = px.line(x=data_coldwar_usa.year.unique()[:-1], y=percent_failures_usa[:-1],
                 labels={'x': 'year', 'y': 'percent of failures'}, title='percentage failure of USA by year')
figure1.show()

figure2 = px.line(x=data_coldwar_ussr.year.unique(), y=percent_failures_ussr,
                 labels={'x': 'year', 'y': 'percent of failures'}, title='percentage failure of USSR by year')
figure2.show()

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches? 

In [ ]:
figure = px.histogram(data_month_replaced, x="year", color="Country")
figure.show()

In [ ]:
countries_by_year = df_data.groupby(by=['year', 'Country']).count()
countries_by_year.tail(20)

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 

In [ ]:
figure = px.histogram(data_month_replaced, x="year", color="Country")
figure.show()

In [ ]:
#VSN USSR was dominant in 1970s and 1980s. CASC was dominant in 2018, 2019, 2020.

In [ ]:
new_countries_by_year = data_month_replaced.groupby(by=['year','Organisation']).count()
new_countries_by_year.head(20)